In [ ]:
!pip install spacy==2.1.0 
!pip install neuralcoref
!pip install flair
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/Kode/NLP_project.zip -d /content
!cp /content/code/sentimentAnalysis.py /content

!cp /content/code/preprocessing_util/utils.py /content


#################################################
#
# Sentiment analysis
#
#################################################
!pip install allennlp
!pip install allennlp_models
!pip install --upgrade google-cloud-storage

!python -m spacy download en_core_web_sm

import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
import re
import string
from itertools import combinations
from collections import Counter


import itertools
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence
import operator
import json
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Kode/NLP_project.zip
replace /content/reports/NLP_report.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 7.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
#tagger = SequenceTagger.load('ner') 
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text


def flair_NER(book):
    """
    flair_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """

    # 00 Pretvori knjigo v stavke:
    sent = sent_tokenize(book)

    # 01 NER model
    entity_dict = []
    for line_num, line in enumerate(tqdm(sent)):

        # 01a Predict:
        sentence = Sentence(line)
        tagger.predict(sentence)

        # 01b Vmesni izpis (brez lokacije):
        #print(sentence.to_tagged_string())
        #print(sentence.get_spans('ner'))

        # 01c Ekstrakcija podatka iz stavka:
        for entity in sentence.get_spans('ner'):        # veliko različnih flair - nerov
            name = entity.text

            # str location
            # start_pos = entity.start_position    # št. str
            # stop_pos = entity.end_position      # št. str

            # token location
            tmp_flag = True
            for token in entity:
                if tmp_flag:
                    start_pos = token.idx -1
                    stop_pos = token.idx
                else:
                    stop_pos = token.idx
                tmp_flag = False

            tag = entity.get_label("ner").value             # tag = entity.tag
            conf_score = entity.get_label("ner").score      # conf_score = entity.score

            info_dict = {}
            info_dict["name"] = name
            info_dict["tag"] = tag
            info_dict["start_pos"] = start_pos
            info_dict["stop_pos"] = stop_pos
            info_dict["line_num"] = line_num

            entity_dict.append(info_dict)

    return entity_dict


def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        if entity["tag"] == "PER":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1), reverse=True)

    return unique_names


def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return open(path, 'w')

In [ ]:
import spacy
import neuralcoref

from utils import readWholeBook, readBookByChapters, getChaptersInOrder
nlp = spacy.load('en_core_web_sm')  # load the model
neuralcoref.add_to_pipe(nlp)

data_path = "/content/data/books/ASongOfIceAndFire/AGOT/chapters/"
results_path = "/content/results/books/ASongOfIceAndFire/AGOT/chapters"
chapters = getChaptersInOrder(data_path)

for chapter in chapters:
    text = read_text(data_path + chapter)
    chapter = chapter.replace(".txt", "")
    doc = nlp(text)
    file = open('/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref_text/' + chapter, 'w')
    file.write(doc._.coref_resolved)
    file.close()
    entitiy_dict = flair_NER(doc._.coref_resolved)
    unique_names = get_names_from_NER(entitiy_dict)

    with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref/" + chapter + "_entity_dict") as f:
        json.dump(entitiy_dict, f)

    with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref/" + chapter + "_unique_names") as f:
        json.dump(unique_names, f)

100%|██████████| 83/83 [00:13<00:00,  6.07it/s]


In [ ]:
from utils import readWholeBook, readBookByChapters, getChaptersInOrder
import numpy as np

from allennlp.predictors.predictor import Predictor
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

data_path = "/content/data/books/ASongOfIceAndFire/AGOT/chapters/"
results_path = "/content/results/books/ASongOfIceAndFire/AGOT/chapters"
chapters = getChaptersInOrder(data_path)

for chapter in chapters:
    text = read_text(data_path + chapter)
    split_text = np.array_split(nltk.sent_tokenize(text), 25)
    
    coref_chapter = ""
    for i in range(25):
        part = ' '.join(split_text[i])   
        pred = predictor.predict(document=part)
        resolved = predictor.coref_resolved(part)
        coref_chapter += resolved
    file = open('/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref_text_allen/' + chapter, 'w')
    file.write(coref_chapter)
    file.close()

    # entitiy_dict = flair_NER(coref_chapter)
    # unique_names = get_names_from_NER(entitiy_dict)

    # chapter = chapter.replace(".txt", "")

    # with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref_allen/" + chapter + "_entity_dict") as f:
    #     json.dump(entitiy_dict, f)

    # with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref_allen/" + chapter + "_unique_names") as f:
    #     json.dump(unique_names, f)